<a href="https://colab.research.google.com/github/AbhiMalappa/NLP/blob/main/helpdesk_BERT_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch


In [ ]:
# funct to get predictions
model_ckpt = "drive/MyDrive/Colab Notebooks/NLP/trained_model/Finetune_BERT_for_helpdesk_issue_classification"
model_ckpt = "drive/MyDrive/Colab Notebooks/NLP/trained_model/V2_Finetune_BERT_for_helpdesk_issue_classification_7_class"

def get_pred(text):

  device = torch.device("cuda")

  #read model
  model = AutoModelForSequenceClassification.from_pretrained(model_ckpt).to(device)

  # from model config get labels names
  labels = model.config.id2label.values()
  labels = list(labels)

  #tokenize input
  tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
  input_encoded = tokenizer(text, return_tensors='pt').to(device)

  # get model pred on tokenized input
  with torch.no_grad():
    outputs = model(**input_encoded) # output is a torch tensor with logit values on gpu

  #output logits
  logits = outputs.logits[0] # get logit values
  #output probs
  probabilities = nn.functional.softmax(logits, dim=-1) # convert logit to prob

  # return a dic with labels and probs
  probabilities = probabilities.cpu().detach().numpy() # convert to numpy on cpu
  result = {labels[i]: probabilities[i] for i in range(len(labels))}

  return result, #probabilities

def pipeline_clf(text):
  device = torch.device("cuda")
  clf = pipeline('text-classification',model = model_ckpt)
  pipeline_clf_label = clf(text)[0]['label']
  pipeline_clf_score = clf(text)[0]['score']
  return pipeline_clf_label, pipeline_clf_score


### Predict on new data

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('drive/MyDrive/Colab Notebooks/training_text_calssifiaction.csv')
df = df.sample(3)

# call prediction
df['pred_label'] = df['string_value'].apply(lambda x: pipeline_clf(x)[0])
df['pred_label_score'] = df['string_value'].apply(lambda x: pipeline_clf(x)[1])
df['pred_val_all_class'] = df['string_value'].apply(lambda x: get_pred(x))

df.head(2)

### Predict on text input

In [ ]:
text = [
    " agent went home early "
    ]

In [ ]:
sentement_obj(text)
#print(f' Top pred - {pipeline_clf(text)}\n')

get_pred(text)

[{'label': 'neutral', 'score': 0.8309993743896484}]

({'agent_unprofessional': 0.058254667,
  'long_holdtime': 0.5411721,
  'multiple_contacts_made': 0.14343858,
  'negative_feedback': 0.11775059,
  'no_resolution': 0.044664945,
  'positive_feedback': 0.083976366,
  'service_portal': 0.0107428245},)

In [ ]:
!pwd

/content


In [ ]:
import pandas as pd
df = pd.read_csv("drive/MyDrive/Colab Notebooks/NLP/split.csv")
df.head()

,Comments
0,It took a few days to close and I am still hav...
1,I had created multiple tickets for RxNotifier ...
2,Of the several different phone numbers and cha...
3,Although I still do not have the proper access...
4,Nidia did not understand what I was asking for...


In [ ]:
df['string_value'] = df['Comments']
df['string_value'] = df['string_value'].str.lower()

In [ ]:
df.drop_duplicates(subset=None, keep='first', inplace=True)
df['string_value'].fillna(' ', inplace=True)

<ipython-input-14-6afe23f4cf0c>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['string_value'].fillna(' ', inplace=True)


In [ ]:
# split string_value - each sentence into seperate rows
from nltk.tokenize import sent_tokenize
import time

t0=time.time()
df['string_value'] = df['string_value'].apply(lambda x : sent_tokenize(x))
s = df.apply(lambda x: pd.Series(x['string_value']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'string_value'
df=df.drop('string_value', axis=1).join(s)
t1=time.time()
print(f"Execution Time= {t1- t0}")

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
from nltk.tokenize import sent_tokenize
text = "sdgsdgs . sdgs dg "

sent_tokenize(text)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
